In [1]:
using Pkg
Pkg.activate("/home/dabajabaza/jianguoyun/Workspace/SmartParser/")

using SmartParser

using Profile

 Activating environment at `~/jianguoyun/Workspace/SmartParser/Project.toml`
┌ Info: Precompiling SmartParser [286a8d42-4b94-434a-8a6f-23eaeb4e0727]
└ @ Base loading.jl:1278


In [2]:
function build_structure_tree(fn)
    lines, lines1, patts, code = load_file(fn) ;
    if length(lines)==0
        return String[], String[], Multiline(), code
    end
    B = build_block_init(patts) ;
    B  = find_block(B) ;
    B  = loop_correct_1_1_until_stable(B) ;
    #B  = find_block(B) ;
    #B  = loop_correct_1_1_until_stable(B) ;
    return lines, lines1, B, code
end

build_structure_tree (generic function with 1 method)

In [3]:
fns = [
    "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test4.pw.x.out" ,
    "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test3.pwdry.x.out" ,
    "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test1.pw.x.out" ,
    "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test2.ph.x.out",
    "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/lammps.log" ,
]

5-element Array{String,1}:
 "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test4.pw.x.out"
 "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test3.pwdry.x.out"
 "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test1.pw.x.out"
 "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/test2.ph.x.out"
 "/home/dabajabaza/jianguoyun/Workspace/SmartParser/test/lammps.log"

In [4]:
Profile.init(n = 10^8, delay = 0.002)
Profile.clear()
@profile  lines, lines1, B, code = build_structure_tree(fns[4]) ;

  0.099379 seconds (172.72 k allocations: 27.502 MiB)


┌ Info: M1.x=206:221 , M.x=206:237 , M.n=2
└ @ SmartParser /home/dabajabaza/jianguoyun/Workspace/SmartParser/src/find_block.jl:105


In [6]:
data = copy( Profile.fetch() ) ;

open("profile.use_copy.txt","w") do f 
    Profile.print(f, data, format=:flat, mincount=400)
end

In [ ]:
DATA = []


for fn in fns
    @info fn
    lines, lines1, B, code = build_structure_tree(fn) ;
    codeinv = revert(code) ;
    BD = parse_file(B, lines, lines1, codeinv) ;
    push!(DATA, (BD, B, code, codeinv, lines, lines1))
    NL = block_print(B, lines1, offset=0, mute=true) ;
    open("$(fn).replaced.txt","w") do f
        write(f,join(NL,"\n"))
    end
end

In [ ]:

function search_kw_in_tree_data(t::Multiline, kw)
    f(x) = ((x isa Multiline) ? [[]] : ([[v for (k,v) in data if k==kw] for data in x.DATA]))
    return [(i,x) for (i,x) in enumerate(collect_action_dfs(t,f)) if unique(x)!=[[]]]
end


function block_by_kw(t::Multiline, kw)
    @inline kw_in_sl(kw,sl) = length([1 for data in sl.DATA for (k,v) ∈ data if k==kw])>0
    h(x) = ((x[2] isa Multiline) 
                ? vcat( # pick the correct block (labelled by 1)
                        [CD[first.(CD).==1] for CD in x[1]]...,  
                        # and itself if there is any children returns [(0,nothing)]
                        (any([kw_in_sl(kw,sl) for sl in children(x[2]) if (sl isa Singleline)]) ? [(1,x[2])] : []) )
                : [] )
    DFS(t, x->nothing, x->nothing, h)
end


function generate_line(patt::Vector{Int}, codeinv::Dict, data)
    if patt==[999999999,]
        @assert unique(first.(data))==["£",]
        return join(string.(last.(data)), "  ")
    else
        line = []
        idata = 1
        Ndata = length(data)
        #@show [codeinv[p] for p ∈ patt]
        for p ∈ patt
            w = codeinv[p]
            @assert  w != "£++++++++++++++++"
            if idata<=Ndata && w==data[idata][1]  # found data,
                #  replace p by data
                push!(line, string(data[idata][2]))  
                idata += 1
            else
                push!(line, w)
            end
        end
        return join(line, "  ")
    end
end


function reconstruct_file(DATATREE::Multiline, codeinv::Dict)
    single_line_reconstr(y) = [generate_line(y[2].p, codeinv, y[2].DATA[i]) for i=1:y[2].n]
    multi_line_reconstr(y)  = vcat(y[1]...)
    line_reconstr(y) = ((y[2] isa Singleline) ? single_line_reconstr(y) : multi_line_reconstr(y))
    DFS(DATATREE, identity, identity, line_reconstr)
end
